
# <font color='red'>**Demo del Pipeline de Cálculo Automático de BALANCES DE ENERGÍA EONTEC**</font>

## 1. Introduccion

Este demo realiza el Cálculo de Los balances de Energía con un programa de despachos y la información real de despacho de los generadores, la primera se obtiene de XM el ente regulador del mercado de energía de Colombia y el segundo, de los reportes periódicos que saca el generador, generalmente son diarios.

Tambien se obtiene la diferencia del ***plan vs lo real***, y el valor de la energía generada ***real, vs la remanente*** que hace falta, y que hay que cubrir con contratos de energía. 

***"Si me sobra vendo la Energía, si me falta, la tengo que comprar"***

### Pasos

### 1. Fuentes (Extract) 


En este proceso la **ETL** se encargará de extraer la información de las siguientes fuentes:

- Descarga Archivo de despacho horario de las centrales de energia consolidado
- Consumo de API de despacho programado para la fecha del ente regulador.
- Consumo de API de Precio de la energia por kWh para el día asignado

### 2. Transformaciones (Transform)

Los archivos obtenidos se transformarán a un formato manejable, se realizarán cruces entre los data sets y filtros

- Cambio de formato de Excel a DataFrame
- Cambio de formato de JSON a DataFrame
- Agregación, cálculos de los balances por unidad de generación (por planta)
- Identificación de operación: Compra o Venta.

### 3. Transferencias o Cargas (Load)

Finalmente se depositará o la disponibilizará para descarga en un destino para que los analistas o el regulador la usen.

- Carga por web a Bucket o Carpeta en la nube del regulador

## Preparación de Paquetes

En esta sección se instalan los paquetes, librerías y componentes que le permitirán al pipeline realizar sus procesos internos.

In [61]:
import numpy as np
import pandas as pd
import openpyxl
import requests
import json
import os

## Declaracion de Variables

Definimos las variables como fecha de inicio y final para realizar el Balance y también el ID para obtener el archivo de despacho. Las variables son:

- **fecha_inicio:** Fecha de inicio del balance
- **fecha_fin:** Fecha fin del balance
- **id_archivo_balance_drive:** Id del archivo publico de despacho de generacion

  **Nota:** la fecha de inicio y la final deben coincidir con la fecha de la generación de energía.


In [62]:
# Define las fechas de inicio y fin
fecha_inicio="2024-07-10"
fecha_fin="2024-07-10"
id_archivo_balance_drive="1kXBIgeXQzM6i1slWQE0vuiPsdgFCSUy2"
api_key_1="r2sbIojfgWlKMJTNBTXw42yDAVYKDspgJnJiP0c3MrCBeTEYv6DfQGjmUjmrPLoC"
api_key_2="YEjIQuxqHPxUqMdWs0cPcga4mBALFWBdWQmOzG6usqO6PuUBl30VXektDO378YlJ"

### A. PRECIOS DE ENERGÍA

Descargamos el API de los **Precios de Energía** estos datos están en formato JSON. Utilizamos la librería de Python llamada Requests y con ella obtenemos el api de la URL, y almacenamos el resultado en una estructura de datos denominada dataframe.

### (E) Extraccion 

In [63]:
# (E) Extraccion del API de Precios de la Energía
url = f"https://www.simem.co/backend-files/api/PublicData?startDate={fecha_inicio}&endDate={fecha_fin}&datasetId=96D56E"

# Realizar la solicitud HTTP, para recibir los datos del sitios web
response = requests.get(url)

# Mostrar el tipo de respuesta de la solicitud HTTP
print("Codigo de Respuesta: ",response)

Codigo de Respuesta:  <Response [200]>


In [25]:
# Mostrar el contenido de Json en los atributos de result y records
print(response.json()['result']['records'])

[{'CodigoVariable': 'PPBO', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TX2', 'Valor': 202.4419}, {'CodigoVariable': 'PPBOGReal', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TX2', 'Valor': 213.6887}, {'CodigoVariable': 'PPBO', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TXF', 'Valor': 208.5071}, {'CodigoVariable': 'PPBOGReal', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TXF', 'Valor': 219.7575}, {'CodigoVariable': 'PPBO', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TXR', 'Valor': 208.5051}, {'CodigoVariable': 'PPBOGReal', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TXR', 'Valor': 219.7562}, {'CodigoVariable': 'PPBOGReal', 'Fecha': '2024-07-10', 'CodigoDuracion': 'P1D', 'UnidadMedida': 'COP/kWh', 'Version': 'TX1', 'Valor': 213.6898}]

### (T) Transformación

In [26]:
# Convertir el JSON en un DataFrame
dfPreciosBolsa = pd.json_normalize(response.json()['result']['records'])

# Mostrar el DataFrame PreciosBolsa
dfPreciosBolsa.head()

,CodigoVariable,Fecha,CodigoDuracion,UnidadMedida,Version,Valor
0,PPBO,2024-07-10,P1D,COP/kWh,TX2,202.4419
1,PPBOGReal,2024-07-10,P1D,COP/kWh,TX2,213.6887
2,PPBO,2024-07-10,P1D,COP/kWh,TXF,208.5071
3,PPBOGReal,2024-07-10,P1D,COP/kWh,TXF,219.7575
4,PPBO,2024-07-10,P1D,COP/kWh,TXR,208.5051


### B. DESPACHO  DE ENERGÍA POR UNIDAD Y CENTRAL.

Descargamos el API de los datos de **despacho de Energía** en formato JSON, utilizamos la librería de Python llamada Requests y con ella obtenemos el api de la URL, y almacenamos el resultado en una estructura de datos denominada dataframe.

### (E) Extraccion 

In [64]:
# (E) Extraccion del Plan de Despacho de Energía por Unidad y Central
url = f"https://www.simem.co/backend-files/api/PublicData?startdate={fecha_inicio}&enddate={fecha_fin}&datasetId=ff027b"

# Realiza la solicitud GET
response = requests.get(url)

# Mostrar el tipo de respuesta de la solicitud HTTP
print(response)

<Response [200]>


### (T) Transformación

In [65]:
# Convierte la respuesta JSON en un DataFrame de Pandas
df_Despachos_Acme = pd.json_normalize(response.json()['result']['records'])

# Renombramos los atributos de Valor y Codigo Planta
df_Despachos_Acme = df_Despachos_Acme.rename(columns={
    'Valor': 'capacidad',
    'CodigoPlanta': 'codigo'
})

# Convertimos la columna 'FechaHora' a formato datetime
df_Despachos_Acme['FechaHora'] = pd.to_datetime(df_Despachos_Acme['FechaHora'])

# Extraemos el día, mes, año y hora de la columna 'FechaHora'
df_Despachos_Acme['dia_despacho'] = df_Despachos_Acme['FechaHora'].dt.day
df_Despachos_Acme['mes_despacho'] = df_Despachos_Acme['FechaHora'].dt.month
df_Despachos_Acme['anio_despacho'] = df_Despachos_Acme['FechaHora'].dt.year
df_Despachos_Acme['hora_despacho'] = df_Despachos_Acme['FechaHora'].dt.hour

# Filtrar el DataFrame según las condiciones especificadas
df_Despachos_Acme = df_Despachos_Acme[df_Despachos_Acme['codigo'].isin(['ZPA2', 'ZPA3', 'ZPA4', 'ZPA5', 'GVIO', 'QUI1', 'CHVR'])]

# Seleccionamos las columnas necesarias
df_Despachos_Acme = df_Despachos_Acme[['codigo', 'anio_despacho', 'mes_despacho', 'dia_despacho', 'hora_despacho', 'capacidad']]
df_Despachos_Acme['capacidad'] = df_Despachos_Acme['capacidad'].astype('int64')

# Mostramos el contenido del dataframe Despachos_Acme
df_Despachos_Acme.head()

,codigo,anio_despacho,mes_despacho,dia_despacho,hora_despacho,capacidad
173,CHVR,2024,7,10,0,0
204,GVIO,2024,7,10,0,60000
260,QUI1,2024,7,10,0,78000
328,ZPA2,2024,7,10,0,17000
329,ZPA3,2024,7,10,0,31000


In [29]:
# reset_index
df_Despachos_Acme = df_Despachos_Acme.reset_index(drop=True)

# Mostramos el contenido del dataframe Despachos_Acme
df_Despachos_Acme.head()

,codigo,anio_despacho,mes_despacho,dia_despacho,hora_despacho,capacidad
0,CHVR,2024,7,10,0,0
1,GVIO,2024,7,10,0,60000
2,QUI1,2024,7,10,0,78000
3,ZPA2,2024,7,10,0,17000
4,ZPA3,2024,7,10,0,31000


### C. GENERACIÓN DE ENERGÍA (CAPACIDAD)

Descargamos de Google Drive el archivo de consolidado de generación, utilizamos Requests y con ella descargamos el archivo de la URL, en este caso tenemos que traer el archivo a una ubicación local, y posteriormente lo cargamos con Pandas.

### (E) Extraccion 

In [66]:
### (E) Descarga del archivo con nuestros datos reales de generación
download_url = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive}'

# Realiza la solicitud GET para descargar el archivo
response = requests.get(download_url)

# Mostrar el tipo de respuesta de la solicitud HTTP
print(response)

<Response [200]>


### (T) Transformación

In [67]:
# Ruta local donde se guardará el archivo descargado
local_file_path = 'balances3.xlsx'

# Guarda el contenido en un archivo local
with open(local_file_path, 'wb') as file:
    file.write(response.content)

# Lee el archivo Excel en un DataFrame de Pandas
ArchivoCapacidad = pd.read_excel(local_file_path, engine='openpyxl', header = 5)

# Muestra el DataFrame ArchivoCapacidad
ArchivoCapacidad.head()

,Unnamed: 0,FECHA,PLANTA,GENERADOR,CAPACIDAD (Kwh),CODIGO
0,NaN,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2
1,NaN,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2
2,NaN,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2
3,NaN,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2
4,NaN,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2


In [68]:
# Seleccionar y renombrar las columnas en el DataFrame original
columnas = ["useless", "fecha", "planta", "generador", "capacidad", "codigo"]
ArchivoCapacidad.columns = columnas

# Extraemos las columnas el año, mes, día y hora 
ArchivoCapacidad['anio'] = ArchivoCapacidad['fecha'].dt.year
ArchivoCapacidad['mes'] = ArchivoCapacidad['fecha'].dt.month
ArchivoCapacidad['dia'] = ArchivoCapacidad['fecha'].dt.day
ArchivoCapacidad['hora'] = ArchivoCapacidad['fecha'].dt.hour

# Muestra el DataFrame ArchivoCapacidad
ArchivoCapacidad.head()

,useless,fecha,planta,generador,capacidad,codigo,anio,mes,dia,hora
0,NaN,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2,2024,7,10,0
1,NaN,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2,2024,7,10,1
2,NaN,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2,2024,7,10,2
3,NaN,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2,2024,7,10,3
4,NaN,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2,2024,7,10,4


In [69]:
# Seleccionamos las columnas que necesitamos para el calculo
ArchivoCapacidad = ArchivoCapacidad[['codigo', 'anio','mes', 'dia', 'hora', 'capacidad']]

# Muestra el DataFrame ArchivoCapacidad
ArchivoCapacidad.head()

,codigo,anio,mes,dia,hora,capacidad
0,ZPA2,2024,7,10,0,33000
1,ZPA2,2024,7,10,1,33000
2,ZPA2,2024,7,10,2,33000
3,ZPA2,2024,7,10,3,33000
4,ZPA2,2024,7,10,4,20834


### D. CÁLCULO DEL BALANCE DE ENERGÍA

Ya tenemos dos dataframes, uno con los datos del despacho que vienen del ente regulador **df_Despachos_Acme** y otro con los datos de excel **ArchivoCapacidad**, en esta sección restamos el despacho programado, de nuestra capacidad por hora, y luego realizamos una sumatoria para sacar el **balance consolidado de energía**.

### (T) Transformación

In [70]:
# (T) Cálculo del balance de energía.
# Realizamos el join entre ambos DataFrames
df_balance_horario = pd.merge(
    ArchivoCapacidad,
    df_Despachos_Acme,
    left_on=['codigo', 'anio', 'mes', 'dia', 'hora'],
    right_on=['codigo', 'anio_despacho', 'mes_despacho', 'dia_despacho', 'hora_despacho'],
    how='inner'
)

# Mostramos el dataset obtenido del Join
df_balance_horario.head()

,codigo,anio,mes,dia,hora,capacidad_x,anio_despacho,mes_despacho,dia_despacho,hora_despacho,capacidad_y
0,ZPA2,2024,7,10,0,33000,2024,7,10,0,17000
1,ZPA2,2024,7,10,1,33000,2024,7,10,1,17000
2,ZPA2,2024,7,10,2,33000,2024,7,10,2,17000
3,ZPA2,2024,7,10,3,33000,2024,7,10,3,17000
4,ZPA2,2024,7,10,4,20834,2024,7,10,4,17000


In [45]:
# Calculamos el balance disponible horario
df_balance_horario['balance_disponible_horario'] = df_balance_horario['capacidad_x'] - df_balance_horario['capacidad_y']

# Mostramos los valores obtenidos del balance
df_balance_horario['balance_disponible_horario'].head()

0    16000
1    16000
2    16000
3    16000
4     3834
Name: balance_disponible_horario, dtype: int64

In [46]:
# Agrupamos por anio, mes, dia, y codigo para sumar los balances disponibles horarios
df_balance_consolidado = df_balance_horario.groupby(['anio', 'mes', 'dia', 'codigo']).agg({
    'balance_disponible_horario': 'sum'
}).reset_index()

# Mostramos el DataFrame balance_consolidado
df_balance_consolidado.head()

,anio,mes,dia,codigo,balance_disponible_horario
0,2024,7,10,GVIO,11506280
1,2024,7,10,QUI1,3607414
2,2024,7,10,ZPA2,201706
3,2024,7,10,ZPA3,-280652
4,2024,7,10,ZPA4,-203312


In [47]:
# Renombramos la columna agregada
df_balance_consolidado = df_balance_consolidado.rename(columns={'balance_disponible_horario': 'consolidado_planta'})

# Mostramos el DataFrame balance_consolidado
print(df_balance_consolidado.head())

   anio  mes  dia codigo  consolidado_planta
0  2024    7   10   GVIO            11506280
1  2024    7   10   QUI1             3607414
2  2024    7   10   ZPA2              201706
3  2024    7   10   ZPA3             -280652
4  2024    7   10   ZPA4             -203312


Ahora con el precio de la bolsa que está en la tabla **Precios Bolsa** vamos a **multiplicar** de la capacidad en el **Balance Consolidado** que acabamos de calcular y eso nos dará el valor de la energía en miles de millones de pesos.

In [71]:
# Filtrar PreciosBolsa con codigo = PPBOGReal y version = TXR
df_precios_filtrados = dfPreciosBolsa[
    (dfPreciosBolsa['CodigoVariable'] == 'PPBOGReal') & 
    (dfPreciosBolsa['Version'] == 'TXR')
]

df_precios_filtrados.head()

,CodigoVariable,Fecha,CodigoDuracion,UnidadMedida,Version,Valor
5,PPBOGReal,2024-07-10,P1D,COP/kWh,TXR,219.7562


In [73]:
# Crear una copia explícita del DataFrame filtrado
df_precios_filtrados = df_precios_filtrados.copy()

# Convertir la columna 'Fecha' a datetime
df_precios_filtrados['Fecha'] = pd.to_datetime(df_precios_filtrados['Fecha'], errors='coerce')

# Extraer componentes de fecha utilizando .loc para evitar advertencias
df_precios_filtrados.loc[:, 'dia'] = df_precios_filtrados['Fecha'].dt.day
df_precios_filtrados.loc[:, 'mes'] = df_precios_filtrados['Fecha'].dt.month
df_precios_filtrados.loc[:, 'anio'] = df_precios_filtrados['Fecha'].dt.year

df_precios_filtrados.head()

,CodigoVariable,Fecha,CodigoDuracion,UnidadMedida,Version,Valor,dia,mes,anio
5,PPBOGReal,2024-07-10,P1D,COP/kWh,TXR,219.7562,10,7,2024


In [74]:
# Seleccionamos las columnas que necesitamos para realizar el calculo
df_precios_seleccionado = df_precios_filtrados[['dia', 'mes', 'anio', 'Valor']]

df_precios_seleccionado.head()

,dia,mes,anio,Valor
5,10,7,2024,219.7562


In [75]:
# Realizar el merge (equivalente al JOIN en SQL)
df_balances_compras_ventas = pd.merge(
    df_balance_consolidado,
    df_precios_seleccionado,
    how='inner',  # Usa 'inner' para replicar el JOIN
    on=['dia', 'mes', 'anio']
)

df_balances_compras_ventas.head()

,anio,mes,dia,codigo,consolidado_planta,Valor
0,2024,7,10,GVIO,11506280,219.7562
1,2024,7,10,QUI1,3607414,219.7562
2,2024,7,10,ZPA2,201706,219.7562
3,2024,7,10,ZPA3,-280652,219.7562
4,2024,7,10,ZPA4,-203312,219.7562


In [76]:
# Calcular la columna Compromisos_MCOP
df_balances_compras_ventas['Compromisos_MCOP ($ MM)'] = (
    df_balances_compras_ventas['consolidado_planta'] * df_balances_compras_ventas['Valor']
) / 1000000

df_balances_compras_ventas['Compromisos_MCOP ($ MM)'].head()

0    2528.576369
1     792.751592
2      44.326144
3     -61.675017
4     -44.679073
Name: Compromisos_MCOP ($ MM), dtype: float64

In [77]:
# Seleccionar las columnas finales
dfReporteCompraVentaEnergiaAcme = df_balances_compras_ventas[
    ['anio', 'mes', 'dia', 'codigo', 'consolidado_planta', 'Compromisos_MCOP ($ MM)']
]

# Crear la columna Operacion basada en la condición
dfReporteCompraVentaEnergiaAcme['Operacion'] = np.where(
    dfReporteCompraVentaEnergiaAcme['Compromisos_MCOP ($ MM)'] < 0,
    "Comprar",
    "Vender"
)

dfReporteCompraVentaEnergiaAcme.head()

,anio,mes,dia,codigo,consolidado_planta,Compromisos_MCOP ($ MM),Operacion
0,2024,7,10,GVIO,11506280,2528.576369,Vender
1,2024,7,10,QUI1,3607414,792.751592,Vender
2,2024,7,10,ZPA2,201706,44.326144,Vender
3,2024,7,10,ZPA3,-280652,-61.675017,Comprar
4,2024,7,10,ZPA4,-203312,-44.679073,Comprar


### E. TRANSFERENCIA DEL ARCHIVO

Ahora tomamos el resultado de las compras y ventas de energía y lo transferimos al destino para que lo cargue el usuario final. Vamos a simular la carga del archivo a un servicio de transferencia de archivos llamado https://fastupload.io
Vamos a **obtener el CSV**, para ello vamos a usar python para **validar si nuestro archivo es un CSV y si es, lo copiamos** a una ruta que podamos acceder.


In [78]:
# Guardar el DataFrame en un archivo CSV
local_csv_path = "dfReporteCompraVentaEnergiaAcme.csv"
dfReporteCompraVentaEnergiaAcme.to_csv(local_csv_path, index=False, header=True)

# Verificar si el archivo se ha guardado correctamente
if os.path.exists(local_csv_path):
    print("--------------------------------------------------------")
    print(f"Archivo guardado en: {local_csv_path}")
    print("--------------------------------------------------------")
else:
    print("------------------------------")
    print("No se pudo guardar el archivo.")
    print("------------------------------")

--------------------------------------------------------
Archivo guardado en: dfReporteCompraVentaEnergiaAcme.csv
--------------------------------------------------------


## *(L)* Autenticación en el sitio remoto del ente regulador

Para poder **cargar archivos**, hay que **iniciar sesión** en la plataforma, para ello se utiliza un API, https://fastupload.io/api#authorize , en el api, la plataforma iniciará sesión y obtendrá un token (secuencia de texto y números) que nos permitirá subir el archivo.

In [79]:
# Autenticación en el sitio remoto del ente regulador

# Variables de autenticación
file_upload_access_token = ""
file_upload_account_id = ""

# Parámetros de la solicitud
params = {'key1': api_key_1, 'key2': api_key_2}

# Realizar la solicitud GET a la API
response = requests.get("https://fastupload.io/api/v2/authorize", params=params)

print("Respuesta de la Solicitud", response)

Respuesta de la Solicitud <Response [200]>


In [80]:
# Parsear la respuesta JSON
json_response = json.loads(response.text)

print(json_response)

{'data': {'access_token': 'tF1w8C0TEqXyRUb2DtTlLT9QmqnxEUaDcgWer2uU0lOShePa6oK8Pgj3JTHiX1VFpP9CQjP016s6QWanFEKgMMsg89ba2pmFW3DX3KMx9ApInGX5LXkG4IW5lKE5nQnj', 'account_id': 67503}, '_status': 'success', '_datetime': '2024-09-06 23:48:21'}


In [81]:
# Manejar la respuesta y extraer los datos necesarios
try:
    file_upload_access_token = json_response["data"]["access_token"]
    file_upload_account_id = json_response["data"]["account_id"]
except KeyError:
    print("Error autenticando y autorizando en el servicio remoto de carga de archivos.")

Así se ve un token de un API

In [82]:
print(file_upload_access_token)

tF1w8C0TEqXyRUb2DtTlLT9QmqnxEUaDcgWer2uU0lOShePa6oK8Pgj3JTHiX1VFpP9CQjP016s6QWanFEKgMMsg89ba2pmFW3DX3KMx9ApInGX5LXkG4IW5lKE5nQnj


## *(L)* Ahora vamos a cargar el Archivo finalmente con el token

Para ello usamos otra API https://fastupload.io/api#file-upload, la cual nos pide algunos parámetros, como el nombre del archivo, el token, la cuenta y el folder, al final, al imprimir la respuesta, veremos que el arhivo fue cargado "File Uploaded".

In [83]:
# Variables necesarias
upload_folder_id = ""
json_response = ""

# Ruta del archivo CSV que se subirá
file_path = "dfReporteCompraVentaEnergiaAcme.csv"

# Abrir el archivo en modo binario para su subida
with open(file_path, "rb") as archivo:
    files = {"upload_file": (archivo.name, archivo)}  # Crear un diccionario para la subida del archivo

    # Datos adicionales que se envían en la solicitud
    data = {
        "access_token": file_upload_access_token,
        "account_id": file_upload_account_id,
        "folder_id": upload_folder_id  # Incluir folder_id si es necesario
    }

    # Realizar la solicitud POST para subir el archivo
    response = requests.post("https://fastupload.io/api/v2/file/upload", files=files, data=data)
  
# Imprimir la respuesta de la solicitud
print(response.text)

{"response":"File uploaded","data":[{"name":"dfReporteCompraVentaEnergiaAcme (21).csv","size":363,"type":"application\/octet-stream","error":null,"url":"https:\/\/fastupload.io\/5fbea05bb17924af","delete_url":"https:\/\/fastupload.io\/5fbea05bb17924af~d?749ef886144ed6c8fcd94bacf1016420","info_url":"https:\/\/fastupload.io\/5fbea05bb17924af~i","delete_type":"DELETE","delete_hash":"749ef886144ed6c8fcd94bacf1016420","hash":"4805f7fcb71d59d7de5e8c0e3e011c94","stats_url":"https:\/\/fastupload.io\/5fbea05bb17924af~s","short_url":"5fbea05bb17924af","file_id":"661617","unique_hash":"557152e1b089e10c437ae07593aa43b8118524e0db92ec5a77f4fbbe9035cd76","url_html":"&lt;a href=&quot;https:\/\/fastupload.io\/5fbea05bb17924af&quot; target=&quot;_blank&quot; title=&quot;View image on Fastupload.io&quot;&gt;View dfReporteCompraVentaEnergiaAcme (21).csv on Fastupload.io&lt;\/a&gt;","url_bbcode":"[url]https:\/\/fastupload.io\/5fbea05bb17924af[\/url]","success_result_html":""}],"_status":"success","_datetim